In [1]:
import praw
import pandas as pd
import datetime as dt
from collections import defaultdict 

In [2]:
# client_id='PERSONAL_USE_SCRIPT_14_CHARS', 
#                      client_secret='SECRET_KEY_27_CHARS ', 
#                      user_agent='YOUR_APP_NAME'
reddit = praw.Reddit(client_id='R30fcxAZLJ7Wyw',
                     client_secret='8Zf87krHYa_zXRXPWEF8hK2Twfs',
                     user_agent='project1',
                     username = 'gbatra',
                     password = "HOLYMOLY1999"
                    )
print(reddit.read_only)

False


In [3]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [4]:
india_subreddit = reddit.subreddit('india')
print(india_subreddit.description)

### We are looking for additional moderators. If you believe you can help, apply [via modmail](https://www.reddit.com/message/compose?to=%2Fr%2Findia)

###[r/India Rules](https://www.reddit.com/r/India/wiki/rules)

###[PushBullet Notifications](https://www.pushbullet.com/channel?tag=redditindia)

###[r/India Discord Server](https://discord.gg/6V29X7f)

###[Scheduled threads](https://www.reddit.com/r/india/comments/b27sx9/scheduled_threads/)

###[r/India twitter](https://twitter.com/redditindia)

#Filters & Hot Topics:

> **Enable**[All](http://www.reddit.com/r/india/#aa)[Non-Political](http://yy.reddit.com/r/india/#yy)[Politics](http://xx.reddit.com/r/india/#xx)[\[R\]eddiquette](http://zz.reddit.com/r/india/#zz)[AMA](https://am.reddit.com/r/india/search?q=flair%3AAMA&sort=new&restrict_sr=on/#am)[Scheduled](https://sd.reddit.com/r/india/search?q=flair%3AScheduled+-title%3ARandom&restrict_sr=on&sort=new&t=all/#sd)[Science & Tech.](http://bb.reddit.com/r/india/#bb)[Self Posts](http://ss.r

In [5]:
# Each subreddit has five different ways of organizing the topics created by redditors: .hot, .new,
# .controversial, .top, and .gilded. 
# You can also use .search("SEARCH_KEYWORDS") to get only results matching an engine search.

In [6]:
# get 10 hot posts from the MachineLearning subreddit
hot_posts = india_subreddit.hot(limit=10)

In [7]:
# test = []
# for post in hot_posts:
#     print("Title: "+ post.title)
#     print("Score: "+ str(post.score))
#     print("ID: "+ str(post.id))
#     print("Subreddit: "+ str(post.subreddit))
#     print("URL: "+ post.url)
#     print("No. of comments: "+ str(post.num_comments))
#     print("Self Text: "+ post.selftext)
#     print("Post Created: "+ str(post.created))
#     test.append([post.title, post.score, post.id])
#     print()
# test

In [8]:
# india_posts = defaultdict(list)
# for post in hot_posts:
#     india_posts['title'].append(post.title)
#     india_posts['score'].append(post.score)
#     india_posts['url'].append(post.url)
#     india_posts['num_comments'].append(post.num_comments)
#     india_posts['text'].append(post.selftext)
#     india_posts['created'].append(post.created)
#     #  a limit of None means that all MoreComments objects will be replaced until there are none left
#     for comment in post.comments:
#         india_posts['comments'].append(comment)

# india_posts

# This gave out an error ' ValueError: arrays must all be same length' when being converted to Dataframe

In [ ]:
india_posts = defaultdict(list)
for post in hot_posts:
    india_posts['title'].append(post.title)
    india_posts['score'].append(post.score)
    india_posts['url'].append(post.url)
    india_posts['num_comments'].append(post.num_comments)
    india_posts['text'].append(post.selftext)
    india_posts['created'].append(post.created)
    #  a limit of None means that all MoreComments objects will be replaced until there are none left
    post.comments.replace_more(limit=None)
    all_comments = ''
    for comment in post.comments:
        all_comments += comment.body
    
    india_posts['comments'].append(comment)

india_posts

defaultdict(list,
            {'title': ['Coronavirus (COVID-19) Megathread - News and Updates - 4',
              'Announcing r/IndiaMeme, our own sub for memes that are related to India. Welcoming all the memers to submit their memes and OC memes!',
              '#OnThisDay in 1975, history was made when India’s first satellite - Aryabhata, was launched from Kapustin Yar in Russia.',
              'Different stages of hair loss in perfect order. Mumbai local train memories.',
              'All the 186 COVID-19 positive cases that were reported yesterday were asymptomatic, they didn’t know they had Coronavirus. This is more worrying: Delhi CM Arvind Kejriwal',
              'Saudi scholar calls for expulsion of Hindu supremacists from the Gulf',
              "Tejasvi Surya deletes tweet on Arab women that read 'they never had orgasm in the last few hundred years'",
              'Rajasthan Congress MLA took back the ration from a woman after she told him that she supports PM Modi.'

In [ ]:
india_postsdf = pd.DataFrame(india_posts)

In [ ]:
_timestamp = india_postsdf["created"].apply(get_date)
india_postsdf = india_postsdf.assign(timestamp = _timestamp)
del india_postsdf['created']

In [ ]:
india_postsdf.to_csv('india_posts.csv', index=False)

In [ ]:
india_posts = pd.read_csv('india_posts.csv')
india_posts.head(10)

,title,score,url,num_comments,text,comments,timestamp
0,Coronavirus (COVID-19) Megathread - News and U...,114,https://www.reddit.com/r/india/comments/g1zi21...,1441,###[Covid-19 Fundraisers & Donation Links](htt...,fnvpgio,2020-04-16 09:30:15
1,"Announcing r/IndiaMeme, our own sub for memes ...",292,https://www.reddit.com/r/india/comments/fzub9j...,87,HELLO YOU NICE PEOPLE. WE GOT REALLY TIRED OF ...,fnk5g1t,2020-04-12 23:48:46
2,"#OnThisDay in 1975, history was made when Indi...",2056,https://i.redd.it/8guugveh2qt41.jpg,63,NaN,fnvhqlt,2020-04-19 20:23:24
3,Different stages of hair loss in perfect order...,2431,https://i.redd.it/ydbmwsa7jpt41.jpg,75,NaN,fnvf28d,2020-04-19 18:36:00
4,All the 186 COVID-19 positive cases that were ...,569,https://twitter.com/ANI/status/125176181030855...,85,NaN,fnv6zqg,2020-04-19 20:13:23
5,Saudi scholar calls for expulsion of Hindu sup...,202,https://www.middleeastmonitor.com/20200417-sau...,48,NaN,fnwj2va,2020-04-20 00:46:55
6,Tejasvi Surya deletes tweet on Arab women that...,96,https://www.ibtimes.co.in/tejasvi-surya-delete...,15,NaN,fnwkcne,2020-04-20 04:40:01
7,Rajasthan Congress MLA took back the ration fr...,301,https://twitter.com/IndiaToday/status/12517392...,15,NaN,fnw1bs9,2020-04-19 21:20:41
8,TIL my In-Laws have stopped taking milk and my...,116,https://www.reddit.com/r/india/comments/g46sqm...,40,"While talking to my mother today on phone, I c...",fnvniis,2020-04-20 01:52:00
9,"India's cases jumped from 14,352 on Apr 17 to ...",641,https://i.redd.it/2mbd2uqbxot41.jpg,211,NaN,fnvizrp,2020-04-19 16:32:40


In [ ]:
india_posts.isnull().sum().sum()

7

In [ ]:
india_posts.dtypes

title           object
score            int64
url             object
num_comments     int64
text            object
comments        object
timestamp       object
dtype: object

In [ ]:
# india_posts['title'] = pd.Series(india_posts['title'], dtype='string')
# india_posts.loc[:,'title'] = india_posts.loc[:, 'title'].astype(str)
# india_posts['title'].apply(str)
# india_posts['title'] = india_posts['title'].astype('|S')
# india_posts['title']

In [ ]:
flairs=['Politics','Non-Political','AskIndia', "[R]eddiquette", 'Science/Technology', 'Policy/Economy', 'Business/Finance', 'Scheduled', 'Sports', 'Food', 'Photography', 'AMA', ]

In [ ]:
data = defaultdict(list)
for flair in flairs:
    search_flair = india_subreddit.search(flair, limit=100)
    for post in search_flair:
        data['flair'].append(flair)
        data['title'].append(post.title)
        data['score'].append(post.score)
        data['url'].append(post.url)
        data['num_comments'].append(post.num_comments)
        data['text'].append(post.selftext)
        data['created'].append(post.created)
        post.comments.replace_more(limit=None)
        all_comments = ''
        for comment in post.comments:
            all_comments += comment.body

        data['comments'].append(comment)
data

In [ ]:
datadf = pd.DataFrame(data)
_timestamp = datadf['created'].apply(get_date)
datadf = datadf.assign(timestamp = _timestamp)
del datadf['created']
datadf.to_csv('dataset.csv', index=False)

In [ ]:
datadf = pd.read_csv('dataset.csv')
datadf.head(10)